In [1]:
import requests
import json
import os
import shutil

In [2]:
collection = 'EAPES-8f3c1f'
nItems = 10000

In [61]:
def dumpCollection(collection, nItems, mode='w'):
    chunkSize=20
    fileInfo = os.path.join(collection, 'collectionInfo.json')
    fileLog = os.path.join(collection, 'collectionLog.json')
    fileData = os.path.join(collection, 'collectionData.json')

    if mode == 'w':
        os.mkdir(collection)
        response = requests.request('get', 'https://api.elrond.com/collections/{}'.format(collection))
        collectionInfo = response.json()
        with open(fileInfo, mode='w', encoding='utf8') as f:
            json.dump(collectionInfo, f, indent=4)
        done = []
        data = dict()
        todo = [i for i in range(0, nItems, chunkSize)]
    elif mode == 'a':
        with open(fileInfo, mode='r', encoding='utf8') as f:
            done = json.load(f, object_hook=list)
        with open(fileData, mode='r', encoding='utf8') as f:
            data = json.load(f)
        todo = [i for i in range(0, nItems, chunkSize) if i not in done]
    else:
        return

    while todo != []:
        todoCopy = todo.copy()
        for i in todoCopy:
            response = requests.request('get', 'https://api.elrond.com/collections/{}/nfts?from={}&size={}'.format(collection, i,chunkSize))
            if response.status_code == 200:
                nftsData = response.json()
                for nft in nftsData:
                    data[nft['metadata']['id']] = nft
                todo.remove(i)
                done.append(i)
        if todoCopy == todo:
            break
    with open(fileData, mode='w', encoding='utf8') as f:
        json.dump(data, f, indent=4)
    with open(fileLog, mode='w', encoding='utf8') as f:
        json.dump(done, f, indent=4)
    return

In [ ]:
dumpCollection(collection, nItems, mode='w')
while True:
    fileData = os.path.join(collection, 'collectionData.json')
    with open(fileData, mode='r', encoding='utf8') as f:
        data = json.load(f)
    if len(data) == nItems:
        break
    dumpCollection(collection, nItems, mode='a')
fileLog = os.path.join(collection, 'collectionLog.json')
os.remove(fileLog)

In [3]:
def dumpImages(collection):
    folderImages = os.path.join(collection, 'images')
    if 'images' not in os.listdir(collection):
        os.mkdir(folderImages)
    fileData = os.path.join(collection, 'collectionData.json')
    with open(fileData, mode='r', encoding='utf8') as f:
        data = json.load(f)
    imgList = os.listdir(folderImages)
    for k, v in data.items():
        if '{}.png'.format(k) not in imgList:
            response = requests.get(v['url'], stream=True)
            if response.status_code == 200:
                with open(os.path.join(folderImages, '{}.png'.format(v['metadata']['id'])), 'wb') as out_file:
                    shutil.copyfileobj(response.raw, out_file)
                del response
    return

In [15]:
dumpImages(collection)